In [2]:
parent_dir = joinpath(@__DIR__, "..")
include(joinpath(parent_dir, "helper_split.jl"))
include(joinpath(parent_dir, "algorithms.jl"))
include(joinpath(parent_dir, "helper_tt.jl"))

using Distributions
using LinearAlgebra
using Plots

In [ ]:
# Setup target
dim = 2
nu = 5
p = dim
α = 2
s(x)  = (1+norm(x)^2)^α
unnorm_target_s(x) = s(x) * (1 + nu^(-1) * transpose(x) * Σ_inv * x)^(-0.5 * (nu + p))
∇U(x) = ((nu + p) / nu) * x / (1 + norm(x)^2/nu)
∇s(x) = 2 * α * x * (1 + norm(x)^2)^(α - 1)
∇U_s(x) = ∇U(x) - ∇s(x) / s(x)
Hessian_bound_alpha = diagm((nu + p) * ones(dim));

 Time changed process implementation

In [ ]:
T = 2 * 10^3
x_init = randn(dim)
v_init = rand((-1,1),dim)
skele = ZigZag(∇U_s, Hessian_bound_alpha, T, x_init,v_init)

delta = 0.1
discretised = discretise_from_skeleton(skele,delta)
pos_discr1 = [discretised[i][1] for i in eachindex(discretised)]
pos_discr2 = [discretised[i][2] for i in eachindex(discretised)]

chain_tt = approximate_timechanged_skele(discretised, s, delta)
speeds = log.([s(discretised[i]) for i in eachindex(discretised)])
limits = max(maximum(pos_discr1),maximum(pos_discr2)) + 10
p_tt = plot(pos_discr1,pos_discr2, label = "", 
                color=:thermal, 
                # color=:darktest,
                line_z = speeds,
                lw = 1.5, 
                # grid=:none, 
                xlabel = L"x_1", ylabel = L"x_2",
                xlims = [-limits,limits], ylims = [-limits,limits],
                )
display(p_tt)
# savefig("heavytailed_tt.pdf")

Comparison with standard process

In [ ]:
Hessian_bound_alpha = diagm((nu + p) * ones(dim))
skele = ZigZag(∇U, Hessian_bound_alpha, T, x_init,v_init)


pos1 = [skele[i].position[1] for i in eachindex(skele)]
pos2 = [skele[i].position[2] for i in eachindex(skele)]
p_std = plot(pos1,pos2, label = "", lc = "red",
                lw = 1, xlabel = L"x_1", ylabel = L"x_2",
                xlims = [-limits,limits], ylims = [-limits,limits],
                )
# savefig("heavytailed_standard_zz.pdf")